# Word2vec

In [1]:
# 親のフォルダのパスを追加
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [2]:
# Movielensのデータの読み込み
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='../data/ml-10M100K/')
movielens = data_loader.load()

In [3]:
import gensim
import logging

movie_content = movielens.item_content.copy()

# tagが付与されていない映画もあるが、genreはすべての映画に付与されている
# tagとgenreを結合したものを映画のコンテンツ情報として似ている映画を探して推薦していく
# tagがない映画に関しては、NaNになっているので、空のリストに変換してから処理をする
movie_content['tag_genre'] = movie_content['tag'].fillna("").apply(list) + movie_content['genre'].apply(list)
movie_content['tag_genre'] = movie_content['tag_genre'].apply(lambda x:set(map(str, x)))

# タグとジャンルデータを使って、word2vecを学習する
tag_genre_data = movie_content.tag_genre.tolist()
model = gensim.models.word2vec.Word2Vec(tag_genre_data, vector_size=100, window=100, sg=1, hs=0, epochs=50, min_count=5)


In [4]:
# animeタグに似ているタグを確認
model.wv.most_similar('anime')

[('studio ghibli', 0.783661425113678),
 ('zibri studio', 0.7655925154685974),
 ('hayao miyazaki', 0.7300279140472412),
 ('miyazaki', 0.7194485664367676),
 ('pelicula anime', 0.7119366526603699),
 ('japan', 0.6259259581565857),
 ('Animation', 0.5910670757293701),
 ('girl', 0.4963359236717224),
 ('wilderness', 0.4636895954608917),
 ('environmental', 0.45580676198005676)]

In [5]:
# Word2vecContentレコメンド
from src.word2vec import Word2vecRecommender
recommender = Word2vecRecommender()
recommend_result = recommender.recommend(movielens)

In [6]:
#  評価
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.011, Recall@K=0.035
